
<style>
/* Better spacing between cells */
div.text_cell_render { line-height: 1.55; font-size: 16px; }
div.input_area        { border: 1px solid #ddd; border-radius: 6px; }
div.output_area       { background: #f9f9f9; border-radius: 6px; }
h1, h2, h3            { font-family: "Segoe UI", sans-serif; margin-top: 1.2em; }
.callout              { padding:1em 1.2em; border-left:5px solid #2e86de;
                        background:#e9f3ff; border-radius:4px; }
</style>


# 📚 Rabbi‑Corpus Preparation (schema‑aware)

*Prepare <code>all_rabbi_clean.jsonl</code> from <code>merged.json</code> files.*

**Pipeline**

1. Load imports & environment paths  
2. Read the schema (`schema_summary.csv`) → build book metadata  
3. Define helpers (HTML stripping, Hebrew gematria, JSON leaf walk)  
4. Iterate each JSON → emit one clean JSON‑Lines record per leaf



# 1. imports

In [1]:
import json, re, html, glob, pathlib, os
from typing import List, Dict, Any, Tuple
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# 2. Paths & files

In [2]:
ROOT_CLTK   = r"C:/Users/yotam/OneDrive/שולחן העבודה/PROJECTS/ChavrutAI/Sefaria-Export-master/cltk-full"
SCHEMA_CSV  = "schema_summary.csv"

SELECTED_RABBI = "Rashi"
JSONL_OUT   = f"{SELECTED_RABBI}.jsonl"

<div class="callout">
<b>Why keep them here?</b>  
Easy to tweak when you move the notebook to another machine.
</div>

# 3.  Locate source JSONs

In [3]:
pat = os.path.join(ROOT_CLTK, "**", SELECTED_RABBI , "**", "Hebrew", "**", "merged.json")
json_paths = glob.glob(pat, recursive=True)

print("✓", len(json_paths), "merged.json files loaded")

✓ 77 merged.json files loaded


In [4]:
# ------------------------------------------------------------------
# 2.  נתיבים כלליים נוספים
# ------------------------------------------------------------------
EXTRA_DIRS = [
    ("Tanakh",  ["Torah", "Prophets", "Writings"]),
    ("Talmud",  [os.path.join("Bavli", f"Seder {s}")
                 for s in ["Kodashim", "Moed", "Nashim",
                           "Nezikin", "Tahorot", "Zeraim"]]),
    ("Mishnah", [f"Seder {s}"
                 for s in ["Kodashim", "Moed", "Nashim",
                           "Nezikin", "Tahorot", "Zeraim"]]),
    ("Midrash", ["Aggadah", "Halakhah"]),
]

extra_paths = []
for root, branches in EXTRA_DIRS:
    for br in branches:
        base = os.path.join(ROOT_CLTK, root, br)
        pattern = os.path.join(base, "**", "merged.json")
        for fp in glob.glob(pattern, recursive=True):
            # סינון: אין Commentary ואין " on "
            if "Commentary" in fp or " on " in fp:
                continue
            extra_paths.append(fp)

print("✓", len(extra_paths), "merged.json files from core corpora loaded")

✓ 358 merged.json files from core corpora loaded


In [5]:
# ------------------------------------------------------------------
# 3.  מיזוג + הסרת כפילויות
#     (dict.fromkeys שומר על סדר ומסלק כפולים)
# ------------------------------------------------------------------
json_paths = list(dict.fromkeys(json_paths + extra_paths))

print("✓", len(json_paths), "merged.json files total after merge")

✓ 435 merged.json files total after merge


# 4. Read schema ➜ build book metadata

In [6]:
schema = pd.read_csv(SCHEMA_CSV)

meta: Dict[str, Dict[str, Any]] = {}
for book, grp in schema.groupby("book_en"):
    grp = grp.sort_values("level")

    labels_he = grp["section_he"].tolist()       # כמו קודם
    labels_en = grp["section_en"].fillna("").tolist()

    # 🔑 מילון – lower‑case כדי לא להיות תלוי רישיות/רווחים
    en2he = {str(en).strip().lower(): he
             for en, he in zip(labels_en, labels_he)
             if en}                              # מתעלמים מערכי EN ריקים

    meta[book] = {
        "book_he"   : grp.iloc[0]["book_he"],
        "labels_he" : labels_he,
        "en2he"     : en2he,                     # ← חדש
    }

In [7]:
schema[schema.book_en == "Iggerot HaRambam"].head()

,book_en,book_he,domain,subpath,level,section_en,section_he,addressType,nodeType,file
16424,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת","חידושי הרמב""ם",1,Khiddushei HaRambam,"חידושי הרמב""ם",Integer,JaggedArrayNode,Iggerot_HaRambam.json
16425,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת","חידושי הרמב""ם",2,Paragraph,פסקה,Integer,JaggedArrayNode,Iggerot_HaRambam.json
16426,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת",איגרת תימן,1,Iggeret Teiman,איגרת תימן,Integer,JaggedArrayNode,Iggerot_HaRambam.json
16427,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת",איגרת תימן,2,Paragraph,פסקה,Integer,JaggedArrayNode,Iggerot_HaRambam.json
16428,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת",מאמר תחיית המתים,1,Maamar Tekhiyat HaMetim,מאמר תחיית המתים,Integer,JaggedArrayNode,Iggerot_HaRambam.json


In [8]:
schema[schema.subpath=="איגרת תימן"]

,book_en,book_he,domain,subpath,level,section_en,section_he,addressType,nodeType,file
16426,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת",איגרת תימן,1,Iggeret Teiman,איגרת תימן,Integer,JaggedArrayNode,Iggerot_HaRambam.json
16427,Iggerot HaRambam,"איגרות הרמב""ם","שו""ת",איגרת תימן,2,Paragraph,פסקה,Integer,JaggedArrayNode,Iggerot_HaRambam.json


# 5.  Helper functions 🔧

In [9]:
def strip_html(t: str) -> str:
    return re.sub(r"\s+", " ", html.unescape(
        BeautifulSoup(t, "html.parser").get_text(" "))).strip()


UNITS  = ["","א","ב","ג","ד","ה","ו","ז","ח","ט"]
TENS   = ["","י","כ","ל","מ","נ","ס","ע","פ","צ"]
HUNDS  = ["","ק","ר","ש","ת","תק","תר","תש","תת","תתק"]
SPECIAL = {15:"ט״ו", 16:"ט״ז"}

def heb_num(n:int)->str:
    """
    Convert 1‑399 to Hebrew gematria letters.
    Handles ט״ו/ט״ז special case, hundreds, tens, units.
    """
    if n in SPECIAL:
        return SPECIAL[n]
    h, rem = divmod(n, 100)
    t, u   = divmod(rem, 10)
    txt = ""
    if h: txt += HUNDS[h]
    if t: txt += TENS[t]
    if u: txt += UNITS[u]
    # גרש/גרשיים
    return txt[:-1] + "״" + txt[-1] if len(txt) > 1 else txt + "׳"


def json_leaves(node, path, out):
    """
    DFS שמגיע לכל Leaf:
      • dict  – עוברים על כל המפתחות (לא רק ספרות)
      • list  – לפי אינדקס
      • אחר   – מצרפים (path, text)
    path יכול להכיל ints (0,1,…) או str ('0_Iggeret…', 'Intro').
    """
    if isinstance(node, dict):
        for k, v in node.items():
            # ▸ נבנה path חדש
            if re.match(r"^\d+$", k):           # "7"
                new_path = path + [int(k)]      # נשמר כ‑int
            else:
                new_path = path + [k]           # כל השאר כ‑str
            json_leaves(v, new_path, out)

    elif isinstance(node, list):
        for i, item in enumerate(node):         # 0‑based
            json_leaves(item, path + [i], out)

    else:
        out.append((path, str(node)))

**Helpers included**

* `strip_html` – remove tags & entities  
* `heb_num` – robust 1‑399 → גימטריה  
* `json_leaves` – depth‑first walk returning all leaf texts

# 6. Main loop

In [10]:
with open(JSONL_OUT, "w", encoding="utf‑8") as fout:
    for jp in tqdm(json_paths, desc="processing"):
        book_en = pathlib.Path(jp).parent.parent.name  # e.g. 'Rashi on Genesis'
        m = meta.get(book_en)
        if m is None:
            print("⚠️  no schema row for", book_en)
            continue

        data = json.load(open(jp, encoding="utf‑8"))["text"]
        leaves: List[Tuple[List[int], str]] = []
        json_leaves(data, [], leaves)

        labels_he = m["labels_he"]
        en2he = m["en2he"]
        
        
        print(f"Processing {m['book_he']}")

        for path, raw in leaves:
            clean = strip_html(raw)
            if not clean:
                continue

            parts = [m["book_he"]]

            for lvl_idx, idx_val in enumerate(path, 1):
                if lvl_idx > len(labels_he):
                    break

                try:
                    # ▸ index מספרי רגיל
                    n = int(idx_val)
                    parts.append(f"{labels_he[lvl_idx-1]} {heb_num(n+1)}")

                except (ValueError, TypeError):
                    
                    key_en = (
                        idx_val.split("_", 1)[1] if isinstance(idx_val, str) and "_" in idx_val
                        else str(idx_val)
                    ).strip().lower()

                    parts.append(en2he.get(key_en, key_en))

            source = ", ".join(parts)
            merged = f"\"{clean}\" ({source})"
            fout.write(json.dumps({"text": merged}, ensure_ascii=False) + "\n")

print("✅ finished – file saved:", JSONL_OUT)

processing:   0%|                                                                              | 0/435 [00:00<?, ?it/s]

Processing רש"י על בראשית רבה


processing:   0%|▎                                                                     | 2/435 [00:01<03:41,  1.95it/s]

Processing רש"י על משנה אבות
Processing רש"י על ערכין


processing:   1%|▍                                                                     | 3/435 [00:01<03:07,  2.31it/s]

...

Processing ספרי דברים


processing: 100%|███████████████████████████████████████████████████████████████████▋| 433/435 [05:17<00:01,  1.50it/s]

Processing ספרי זוטא
Processing ספרי זוטא


processing: 100%|████████████████████████████████████████████████████████████████████| 435/435 [05:17<00:00,  1.37it/s]

✅ finished – file saved: Rashi.jsonl


In [13]:
import itertools, json, pprint
with open(JSONL_OUT, encoding="utf‑8") as f:
    sample = [json.loads(l) for l in itertools.islice(f, 1)]
pprint.pp(sample, width=120)

[{'text': '"אמון פדגוג. כמא דתימא כאשר ישא האומן את היונק ומתרגמינן בירושלמי כמה דסבל פדגוגא ית ינוקא:" (רש"י על '
          'בראשית רבה, פרק א׳, פסקה א׳, פירוש א׳)'}]


In [14]:
with open(JSONL_OUT, encoding="utf‑8-sig") as f:
    data = [json.loads(l) for l in f]

In [16]:
len(data)

439486

In [17]:
data[0:2]

[{'text': '"אמון פדגוג. כמא דתימא כאשר ישא האומן את היונק ומתרגמינן בירושלמי כמה דסבל פדגוגא ית ינוקא:" (רש"י על בראשית רבה, פרק א׳, פסקה א׳, פירוש א׳)'},
 {'text': '"אמון מוצנע. כמה דתימא ויהי אומן את הדסה (אסתר ב ז\') וכן הפתרון בהשמע דבר המלך ודתו להקבץ את כל נערה בתולה והיה מרדכי מצניע את אסתר שלא תהא מחוללת לאותו רשע ומלאך בא ומגלה אותה הוא שיסד הקליר (ביוצר לפורים) את אסתר גל ממסתור לגאול ממה שהיה מרדכי מסתירה:" (רש"י על בראשית רבה, פרק א׳, פסקה א׳, פירוש ב׳)'}]